# Proves de com llegir els metros

In [136]:
import pandas as pd
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib widget

from dataclasses import dataclass
from typing import Optional, TextIO, List, Tuple

Llegim les dades

In [82]:
estacions = pd.read_csv("data/estacions.csv")
accessos = pd.read_csv("data/accessos.csv")

In [83]:
estacions.head()

,FID,ID_ESTACIO_LINIA,CODI_ESTACIO_LINIA,ID_GRUP_ESTACIO,CODI_GRUP_ESTACIO,ID_ESTACIO,CODI_ESTACIO,NOM_ESTACIO,ORDRE_ESTACIO,ID_LINIA,...,ID_TIPUS_ACCESSIBILITAT,NOM_TIPUS_ACCESSIBILITAT,ID_TIPUS_ESTAT,NOM_TIPUS_ESTAT,DATA_INAUGURACIO,DATA,COLOR_LINIA,PICTO,PICTO_GRUP,GEOMETRY
0,ESTACIONS_LINIA.fid--64d2c1cd_17fdec435f4_43e3,167,111,111,6660111,81,111,Hospital de Bellvitge,1,4,...,1,Accessible,1,Operatiu,1989-10-18,2022-03-31,CE1126,L1,L1,POINT (2.107241921905464 41.344677306491334)
1,ESTACIONS_LINIA.fid--64d2c1cd_17fdec435f4_43e4,119,112,112,6660112,33,112,Bellvitge,2,4,...,1,Accessible,1,Operatiu,1989-10-18,2022-03-31,CE1126,L1,L1,POINT (2.1109169952642675 41.35097554129953)
2,ESTACIONS_LINIA.fid--64d2c1cd_17fdec435f4_43e5,120,113,113,6660113,34,113,Av. Carrilet,3,4,...,1,Accessible,1,Operatiu,1987-04-24,2022-03-31,CE1126,L1,L1,POINT (2.102631528901838 41.35855331385863)
3,ESTACIONS_LINIA.fid--64d2c1cd_17fdec435f4_43e6,168,114,114,6660114,82,114,Rambla Just Oliveras,4,4,...,1,Accessible,1,Operatiu,1987-04-24,2022-03-31,CE1126,L1,L1,POINT (2.0997493392175293 41.3640899130925)
4,ESTACIONS_LINIA.fid--64d2c1cd_17fdec435f4_43e7,121,115,115,6660115,35,115,Can Serra,5,4,...,1,Accessible,1,Operatiu,1987-04-24,2022-03-31,CE1126,L1,L1,POINT (2.1027575190099883 41.367694011134795)


In [84]:
#accessos.head()
estacions.columns


Index(['FID', 'ID_ESTACIO_LINIA', 'CODI_ESTACIO_LINIA', 'ID_GRUP_ESTACIO',
       'CODI_GRUP_ESTACIO', 'ID_ESTACIO', 'CODI_ESTACIO', 'NOM_ESTACIO',
       'ORDRE_ESTACIO', 'ID_LINIA', 'CODI_LINIA', 'NOM_LINIA', 'ORDRE_LINIA',
       'ID_TIPUS_SERVEI', 'DESC_SERVEI', 'ORIGEN_SERVEI', 'DESTI_SERVEI',
       'ID_TIPUS_ACCESSIBILITAT', 'NOM_TIPUS_ACCESSIBILITAT', 'ID_TIPUS_ESTAT',
       'NOM_TIPUS_ESTAT', 'DATA_INAUGURACIO', 'DATA', 'COLOR_LINIA', 'PICTO',
       'PICTO_GRUP', 'GEOMETRY'],
      dtype='object')

## Versio de proves de la classe estació

In [179]:
#We filter stations
columns_of_interest = ['ID_ESTACIO','CODI_GRUP_ESTACIO','NOM_ESTACIO','NOM_LINIA','ID_LINIA','ORDRE_ESTACIO','ID_TIPUS_ACCESSIBILITAT','GEOMETRY']

estacions = estacions[columns_of_interest]
#estacions['GEOMETRY'] = list(map(float,(estacions['GEOMETRY'].split('(')[1].split(')')[0].split())))

In [147]:
Point: Tuple[int,int]
    
@dataclass
class Station:
    st_id: str
    group_code: str
    name: str #A LA VERSIO FINAL TREURE IUSAR UN DICCIONARI SI FA FALTA (ESTALVIAR MEMORIA)
    line_name: str
    line_id: int
    line_order: int
    accessibility: str
    pos: Point
    connections: list[str]
    accesses: list[str]
    line_changes: list[str]
    #def __init__(self, row):
    def __hash__(self):
        return st_id


Creem la llista d'estacions

In [180]:
#Oordenem per linia i ordre dins la linia
estacions = estacions.sort_values(['ID_LINIA','ORDRE_ESTACIO']) #NO FA FALTA CREC

prev: Optional[int] = None
next: Optional[int] = None
station_list : list[Station] = []
#AIXO ES PODRIA MILLORAR AMB SHIFT -> D LA MANERA ACTUAL ES MOLT MOLT POC EFICIENT
for index,row in estacions.iterrows():
    connections: list[str] = []
    if index > 0:
        if estacions.loc[index-1,"ID_LINIA"] == estacions.loc[index,"ID_LINIA"]:
            connections.append(estacions.loc[index-1,"ID_ESTACIO"])
    if index < len(estacions)-1:
        if estacions.loc[index+1,"ID_LINIA"]==estacions.loc[index,"ID_LINIA"]:
            connections.append(estacions.loc[index+1,"ID_ESTACIO"])
    #FALTA COM SABER QUAN HI HA TRANSBORD - UNA OPCIO ES FERHO UN COP SHA LLEGIT JA TOT
    row['GEOMETRY']=list(map(float,(p.split('(')[1].split(')')[0].split())))
    new_station = Station(*(row[columns_of_interest].to_list()),connections,[],[])#, connections[],[],[])
    station_list.append(new_station)
    
    

Creem el graf

In [197]:
Metro = nx.Graph()
Metro.add_nodes_from(range(0,len(station_list)),data = station_list)

In [198]:
mynodes = list(Metro.nodes())
print(mynodes)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170]


In [150]:
p="POINT (2.107241921905464 41.344677306491334)"

In [176]:
x= list(map(float,(p.split('(')[1].split(')')[0].split())))

In [177]:
x

[2.107241921905464, 41.344677306491334]